In [ ]:
import json 
import pandas as pd
from confluent_kafka import Producer

KAFKA_BROKER = "localhost:9092"
TOPIC_NAME = "tbl_product2"

Producer_conf = {'bootstrap.servers': KAFKA_BROKER}
Producer = Producer(Producer_conf)

df_product = pd.read_csv("df_product.csv")
product_data = []

for _, row in df_product.iterrows():
    modified_time = None if pd.isna(row["modified_time"]) else row["modified_time"]

    payload = {
        "id": row["id"],
        "product_name": row["product_name"],
        "category": row["category"],
        "sub_category": row["sub_category"],
        "price": row["price"],
        "profit": row["profit"],
        "stock": row["stock"],
        "created_time": row["created_time"],
        "modified_time": modified_time
    }

    schema = {
        "schema": {  # Perbaikan dari "shema" ❌
            "fields": [
                {"type": "int32", "optional": False, "field": "id"},
                {"type": "string", "optional": False, "field": "product_name"},
                {"type": "string", "optional": False, "field": "category"},
                {"type": "string", "optional": False, "field": "sub_category"},
                {"type": "int32", "optional": False, "field": "price"},
                {"type": "int32", "optional": False, "field": "profit"},
                {"type": "int32", "optional": False, "field": "stock"},
                {"type": "string", "optional": False, "field": "created_time"},
                {"type": "string", "optional": True, "field": "modified_time"}
            ],
            "optional": False,
            "name": "tbl_product",
            "type": "struct"
        },
        "payload": payload
    }
    product_data.append(schema)

def delivery_report(err, msg):
    if err is not None:
        print(f"❌ Gagal mengirim pesan: {err}")
    else:
        print(f"✅ Data dikirim ke Kafka: {msg.topic()} [{msg.partition()}]")

for record in product_data:
    Producer.produce(TOPIC_NAME, json.dumps(record).encode("utf-8"), callback=delivery_report)

Producer.flush()


In [2]:
import pandas as pd

In [7]:
df_order_status = pd.read_csv("df_order_status.csv")
df_order_status

,id,name,created_time
0,2,done,2025-03-03
1,1,ongoing,2025-03-03
